In [4]:
from scrapy.item import Field
from scrapy.item import Item
from scrapy.spiders import Spider
from scrapy.selector import Selector
from scrapy.loader.processors import MapCompose
from scrapy.loader import ItemLoader
from scrapy.crawler import CrawlerProcess

In [5]:
class Noticia(Item) :
    descripcion = Field()
    tag = Field()
    titular = Field()

In [9]:
class LaTerceraSipider(Spider) :
    # Nombre del spider
    name = "LaTerceraSipider"
    # Configuración del USER AGENT en scrapy
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36'
    }
    # URL SEMILLA
    start_urls = ['https://www.latercera.com/lo-ultimo/']
    
    # Función que se llama para realziar el requerimiento de la URL semilla
    def parse(__self__, response) :
        # Selector que extrae los datos de la URL semilla
        sel = Selector(response) 
        # Selector de noticias de la seccióm Ultimas Noticias de La Tercera
        noticias = sel.xpath('//div[@class="archive-list | subgrid"]//article')
        # Recorrer cada noticia
        for noticia in noticias :
            # Instancia del Item con el selector para llenar los datos de la clase Noticia
            item = ItemLoader(Noticia(), noticia)
            # Escritura de las propiedades de la clase Noticia
            item.add_xpath('descripcion', './/p/text()')
            item.add_xpath('tag', './/a/span/text()')
            item.add_xpath('titular', './/h3/a/text()')
            
            # Escritura de los resultados en el archivo de salida
            yield item.load_item()

In [7]:
process = CrawlerProcess({
    'FEED_FORMAT' : 'csv',
    'FEED_URI' : 'results/latercera.csv'
})

2022-04-09 18:29:22 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-04-09 18:29:22 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.2.0, Python 3.9.0 (tags/v3.9.0:9cf6752, Oct  5 2020, 15:34:40) [MSC v.1927 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.4.4, Platform Windows-10-10.0.22000-SP0


In [10]:
process.crawl(LaTerceraSipider)
process.start()

2022-04-09 18:30:52 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, '
               'like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 '
               'Safari/537.36'}
2022-04-09 18:30:52 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-04-09 18:30:52 [scrapy.extensions.telnet] INFO: Telnet Password: a581d6d5ece23913
2022-04-09 18:30:52 [py.warnings] WARNING: C:\Users\smena\AppData\Local\Programs\Python\Python39\lib\site-packages\scrapy\extensions\feedexport.py:289: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)

2022-04-09 18:30:52 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'sc